In [1]:
!pip install transformers torch sentencepiece
!pip install openai

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Create OpenAI client (reads API key from environment)
client = OpenAI()

In [5]:
from transformers import pipeline

# Load BART for summarization
#bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

# Load FLAN-T5 for instruction following
flan_agent = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)

Device set to use cpu


In [7]:

# Agent 1: Force third-person rewording by giving an example
def summary_agent(user_info):
    prompt = (
        "Instruction: Rewrite the following description in third person, "
        "summarizing the key details in a professional, factual style.\n\n"
        "Example:\n"
        "Input: I am a 35-year-old man looking to lose 20 pounds. I work in tech, exercise occasionally, and love pizza.\n"
        "Output: A 35-year-old man working in tech wants to lose 20 pounds. He exercises occasionally and enjoys eating pizza.\n\n"
        "Input:\n"
        f"{user_info}\n\n"
        "Output:"
    )
    summary = flan_agent(prompt, max_length=80, do_sample=False)[0]['generated_text']
    return summary

# Agent 2: Personalized, diverse beginner tips
def tips_agent(user_info):
    prompt = (
        f"This person is trying to lose weight and improve their health:\n\n"
        f"{user_info}\n\n"
        "Suggest three specific, beginner-friendly first steps they can start taking today to build better habits. "
        "Format them clearly as Tip 1, Tip 2, and Tip 3."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a supportive and practical health coach."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content.strip()

# Agent 3: Creative, balanced meal suggestion
def meal_agent(user_info):
    prompt = (
        f"This person is trying to lose weight and eat healthier:\n\n"
        f"{user_info}\n\n"
        "Suggest one healthy, balanced meal they could eat today. "
        "Make sure it includes protein, vegetables, and fiber. "
        "Format as a full meal idea like you're recommending it to a friend."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a registered dietician who gives friendly and realistic meal suggestions."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content.strip()

    return response.choices[0].message.content.strip()
# Mini-Polisher Agent: Rephrase first-person to third-person
def polisher_agent(text_to_polish):
    prompt = (
        "Instruction: Rewrite the following description into third person. "
        "Make it sound professional and objective. Correct any casual language if needed.\n\n"
        f"Input:\n{text_to_polish}\n\n"
        "Output:"
    )
    polished_text = flan_agent(prompt, max_length=80, do_sample=False)[0]['generated_text']
    return polished_text

def weekly_plan_agent(summary_text):
    prompt = (
        f"This person is trying to lose weight and improve their health: {summary_text}\n\n"
        "Create a simple, motivational 7-day health plan. Each day should include one small but meaningful action to help them build healthy habits and lose weight.\n\n"
        "Format your answer like this:\n"
        "Day 1: ...\nDay 2: ...\n...\nDay 7: ..."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a friendly and supportive health coach."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content


In [9]:
def dietician_multi_agent_system(user_info):
    # Step 1: Summarize user input
    rough_summary = summary_agent(user_info)
    
    # Step 2: Polish summary (optional, we know FLAN base isn't perfect)
    polished_summary = rough_summary  # Skipping polishing for now

    # Step 3: Generate tips
    tips = tips_agent(polished_summary)
    
    # Step 4: Suggest a meal
    meal = meal_agent(polished_summary)

    # Step 5: Create a weekly healthy living plan
    weekly_plan = weekly_plan_agent(polished_summary)

    return (
        f" Summary:\n{polished_summary}\n\n"
        f" First Steps:\n{tips}\n\n"
        f" Meal Suggestion:\n{meal}\n\n"
        f" Weekly Healthy Living Plan:\n{weekly_plan}"
    )

In [19]:
user_info = """
I'm a 23-year-old woman. I weigh about 160 lbs and my goal is to lose 15 pounds over the next 3 months. 
I work at a desk job, go to the gym 2-3 times a week, struggle with late-night snacking, and love chocolate. 
I usually skip breakfast and tend to overeat at night.
"""

print(dietician_multi_agent_system(user_info))

 Summary:
I weigh about 160 lbs and my goal is to lose 15 pounds over the next 3 months. I work at a desk job, go to the gym 2-3 times a week, struggle with late-night snacking, and love chocolate. I usually skip breakfast and tend to overeat at night.

 First Steps:
Weight loss: Increase cardio exercise Cardio exercise: Improve sleep Cardio exercise: Increase aerobic exercise Exercise to lose weight

 Meal Suggestion:
Stir-fried tofu with mixed vegetables and brown rice

 Weekly Healthy Living Plan:
Day 1: Start your day with a healthy breakfast to kickstart your metabolism. Try a smoothie with fruits, veggies, and protein powder.

Day 2: Pack a nutritious snack to have at work to avoid late-night snacking. Opt for a handful of nuts or chopped veggies.

Day 3: Practice mindful eating at dinner by chewing slowly and savoring each bite. This can help prevent overeating.

Day 4: Swap out one of your gym sessions for a fun outdoor activity like hiking or biking to keep your workouts inter